FASTAPI

In [ ]:
from fastapi import FastAPI
import sqlite3
import pandas as pd
import joblib

# Charger le modèle de prédiction
model = joblib.load("../model/pkl/linear_model.pkl")  # Remplacez par le chemin de votre modèle

# Créer une instance de FastAPI
app = FastAPI()

# Chemin vers la base de données SQLite
DB_PATH = "../data/power_consumption.db"

#ENDPOINTS CREES
# quelques endpoints classique
#
# Endpoint root
@app.get("/")
def read_root():
    return {"message": "Bienvenue sur l'API de prédiction de consommation électrique"}

# Endpoint pour récupérer des données depuis SQLite (10 premiers)
@app.get("/data/")
def get_data(limit: int = 10):
    conn = sqlite3.connect(DB_PATH)
    query = f"SELECT * FROM power_consumption LIMIT {limit}"
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df.to_dict(orient="records")

# Endpoint pour effectuer une prédiction
@app.get("/predict/")
def predict(
    Temperature: float,
    Humidity: float,
    WindSpeed: float,
    GeneralDiffuseFlows: float,
    DiffuseFlows: float
):
    # Préparer les données d'entrée pour le modèle
    input_data = [[Temperature, Humidity, WindSpeed, GeneralDiffuseFlows, DiffuseFlows]]
    prediction = model.predict(input_data)
    return {"prediction": prediction[0]}


STREAMLIT

In [ ]:
import streamlit as st
import requests

# Configuration de l'application
st.title("Prédiction de Consommation Électrique - Tétouan - MAROC")
st.write("Cette application utilise un modèle de machine learning pour prédire la consommation électrique.")

# Entrée utilisateur pour les prédictions
Temperature = st.number_input("Température", value=6.5)
Humidity = st.number_input("Humidité (%)", value=75.0)
WindSpeed = st.number_input("Vitesse du vent", value=0.08)
GeneralDiffuseFlows = st.number_input("Débit fluide souterrain general", value=0.05)
DiffuseFlows = st.number_input("Débit fluide", value=0.12)

if st.button("Prédire"):
    # Appeler l'API FastAPI pour prédiction
    response = requests.get(
        "http://127.0.0.1:8000/predict/",
        params={
            "Temperature": Temperature,
            "Humidity": Humidity,
            "WindSpeed": WindSpeed,
            "GeneralDiffuseFlows": GeneralDiffuseFlows,
            "DiffuseFlows": DiffuseFlows,
        },
    )
    if response.status_code == 200:
        prediction = response.json()["prediction"]
        st.success(f"La consommation prévue est de : {prediction:.2f}")
    else:
        st.error("Erreur lors de l'appel à l'API")

# Affichage des données brutes
st.write("### Données disponibles")
limit = st.slider("Nombre de lignes à afficher", min_value=1, max_value=50, value=10)
data_response = requests.get(f"http://127.0.0.1:8000/data/?limit={limit}")
if data_response.status_code == 200:
    st.write(data_response.json())
else:
    st.error("Erreur lors de la récupération des données")
